# test notebook

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from src.kernels import gaussian_kernel,laplacian_kernel
from src.model import KernelRegressor
from src.get_data import get_mnist,inject_label_noise
from sklearn.model_selection import train_test_split
from src.metrics import accuracy
import torch

In [4]:
data,target=get_mnist(n_samples=50000,flatten=True)

X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)

In [5]:
print(data.shape)
print(target.shape)
print(target[:4])

torch.Size([50000, 784])
torch.Size([50000])
tensor([4, 0, 9, 2])


In [6]:
print("Training samples:",X_train.shape[0])
print("Test samples:",X_test.shape[0])
print("\nFeature shape:",X_train.shape)
print("Labels shape:",y_train.shape)

Training samples: 40000
Test samples: 10000

Feature shape: torch.Size([40000, 784])
Labels shape: torch.Size([40000])


In [ ]:
kernel_reg=KernelRegressor(kernel_func=gaussian_kernel,sigma=5.0)
kernel_reg.set_train_data(X_train,y_train)
kernel_reg.invers_fit(lr=1e-3,n_iters=1000)

print("Kernel matrix shape:",kernel_reg.K.shape)
print("Alpha coefficient shape:", kernel_reg.alpha.shape)

rkhs_norm=kernel_reg.compute_rkhs_norm()
print("RKHS norm of the learned function f*:",rkhs_norm.item())


Kernel matrix shape: torch.Size([40000, 40000])
Alpha coefficient shape: torch.Size([40000, 10])
RKHS norm of the learned function f*: 170.69827270507812


In [ ]:
predictions=kernel_reg.predict(X_train)
accuracy_train=accuracy(predictions,y_train)

print("Train accuracy:",accuracy_train)

Train accuracy: 1.0


In [ ]:
predictions=kernel_reg.predict(X_test)
accuracy_test=accuracy(predictions,y_test)

print("Test accuracy:",accuracy_test)


Test accuracy: 0.9713000059127808


In [ ]:
noise_levels=[0.01,0.1]

for noise in noise_levels:
    noisy_y_train=inject_label_noise(y_train,noise_ratio=noise)
    kernel_reg.set_train_data(X_train,noisy_y_train)
    kernel_reg.invers_fit()
    predictions=kernel_reg.predict(X_test)
    acc=accuracy(predictions,y_test)
    print(f"Noise level: {noise}, Test accuracy: {acc:.4f}")
    print(f"RKHS norm:{kernel_reg.compute_rkhs_norm().item()}\n")
    

Noise level: 0.01, Test accuracy: 0.9706
RKHS norm:171.625

Noise level: 0.1, Test accuracy: 0.9625
RKHS norm:179.23141479492188



# Study of classifier

Comparison for Laplacian and Gaussian kernels
